In [848]:
#!pip install qutip
#!pip install git+https://github.com/qutip/qutip-qip

In [1]:
import numpy as np
import sys
from scipy.stats import entropy
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import matplotlib.font_manager as font_manager 
from matplotlib.pyplot import gca
import qutip as qt
from qutip_qip.operations import cnot, hadamard_transform
import scipy as sc
path_main = '/Users/stepanvinckevich/Desktop/IMPORTANT NOW/QIS QRL/CODE/QRC/'
path_EVAL = path_main + 'EVALUATION/'
path_EVAL_NOBRIDGE = path_EVAL + 'NOBRIDGE/'

path_CHAOS_SIGNAL = path_EVAL_NOBRIDGE + 'CHAOS/'
path_SIGNAL = path_EVAL_NOBRIDGE + 'NOISY/'

sys.path.append(path_main)

#np.random.seed(10)
np.random.seed(20)

In [1379]:
# Number of qubits in reservoirs A and B
n_qubitsA = 3
k_power_A = 0

n_qubitsB = 3
k_power_B = 0

# Evolution timestep
timestep = 0.3
# Trace preserving tolerance
tp_tol = 1e-5

n_qubits_total = n_qubitsA + n_qubitsB

In [1380]:
# Hamiltonian of A
if k_power_A != 0:
    hamiltonianA = qt.Qobj(np.load(f'/content/gdrive/My Drive/QRC/hamiltonians/hamiltonian{n_qubitsA}_{k_power_A}.npy'), dims=[[2]*n_qubitsA, [2]*n_qubitsA])
else:
    hamiltonianA = qt.Qobj(np.load(path_main + f'/hamiltonians/hamiltonian{n_qubitsA}.npy'), dims=[[2]*n_qubitsA, [2]*n_qubitsA])
identityA = qt.identity(2**n_qubitsA)
identityA.dims = [[2]*n_qubitsA, [2]*n_qubitsA]

# Hamiltonian of B
if k_power_B != 0:
    hamiltonianB = qt.Qobj(np.load(f'/content/gdrive/My Drive/QRC/hamiltonians/hamiltonian{n_qubitsB}_{k_power_B}.npy'), dims=[[2]*n_qubitsB, [2]*n_qubitsB])
else:
    hamiltonianB = qt.Qobj(np.load(path_main + f'/hamiltonians/hamiltonian{n_qubitsB}.npy'), dims=[[2]*n_qubitsB, [2]*n_qubitsB])
identityB = qt.identity(2**n_qubitsB)
identityB.dims = [[2]*n_qubitsB, [2]*n_qubitsB]

# Total hamiltonian
hamiltonian = qt.tensor(hamiltonianA, identityB) + qt.tensor(identityA, hamiltonianB)
# hamiltonian = hamiltonianA

In [1381]:
# Evolution step propagator
propagator = qt.propagator(hamiltonian, timestep)
propagator_dag = propagator.dag().full()
propagator = propagator.full()

# Evolution of entanglement sites a and b (now taken as identity)
prop_a = np.eye(2)
prop_b = np.eye(2)

# Operations
cnot_mtx = cnot(2, control=0, target=1).full()
hadamard = hadamard_transform().full()
cnot_mtx_backward = cnot(2, control=1, target=0).full()

# Entanglement channel
#ent_transform = cnot_mtx @ (np.kron(hadamard @ prop_a, prop_b))
ent_transform = cnot_mtx @ cnot_mtx_backward @ cnot_mtx @ (np.kron(hadamard @ prop_a, prop_b))

ent_transform_dag = np.kron(np.eye(2**(n_qubitsA - 1)), np.kron(ent_transform.T.conj(), np.eye(2**(n_qubitsB - 1))))
ent_transform = np.kron(np.eye(2**(n_qubitsA - 1)), np.kron(ent_transform, np.eye(2**(n_qubitsB - 1))))

#################################################

b = np.random.uniform(0,1/3)
c = np.random.uniform(0,1/3)

P0 = qt.basis(2,0) 
P1 = qt.basis(2,1)


P00 = qt.tensor(P0,P0)
P01 = qt.tensor(P0,P1)
P10 = qt.tensor(P1,P0)
P11 = qt.tensor(P1,P1)

psi00 = b * P00 + np.sqrt(1 - b**2) * P11 
B00 = psi00 * psi00.dag()
B00 = B00.full()

psi11 = -b * P11 + np.sqrt(1 - b**2) * P00 
B11 = psi11 * psi11.dag()
B11 = B11.full()

psi01 = c * P01 + np.sqrt(1 - c**2) * P10 
B01 = psi01 * psi01.dag()
B01 = B01.full()

psi10 = -c * P10 + np.sqrt(1 - c**2) * P01 
B10 = psi10 * psi10.dag()
B10 = B10.full()

PB00_dag = np.kron(np.eye(2**(n_qubitsA - 1)), np.kron(B00.T.conj(), np.eye(2**(n_qubitsB - 1))))
PB00 = np.kron(np.eye(2**(n_qubitsA - 1)), np.kron(B00, np.eye(2**(n_qubitsB - 1))))

PB11_dag = np.kron(np.eye(2**(n_qubitsA - 1)), np.kron(B11.T.conj(), np.eye(2**(n_qubitsB - 1))))
PB11 = np.kron(np.eye(2**(n_qubitsA - 1)), np.kron(B11, np.eye(2**(n_qubitsB - 1))))

PB01_dag = np.kron(np.eye(2**(n_qubitsA - 1)), np.kron(B01.T.conj(), np.eye(2**(n_qubitsB - 1))))
PB01 = np.kron(np.eye(2**(n_qubitsA - 1)), np.kron(B01, np.eye(2**(n_qubitsB - 1))))

PB10_dag = np.kron(np.eye(2**(n_qubitsA - 1)), np.kron(B10.T.conj(), np.eye(2**(n_qubitsB - 1))))
PB10 = np.kron(np.eye(2**(n_qubitsA - 1)), np.kron(B10, np.eye(2**(n_qubitsB - 1))))



######################


In [1382]:
PB00_dag @ PB00 + PB01_dag @ PB01  + PB10_dag @ PB10 + PB11_dag @ PB11

array([[1.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       ...,
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 1.+0.j]])

In [1383]:
# Initial density matrix
init_state = qt.tensor(*[qt.rand_ket(2) for _ in range(n_qubits_total)])
init_state = init_state * init_state.dag()
init_state = init_state.full()

# np.save(f"TF/init_state{n_qubits}.npy", init_state)
# init_state = np.load(f"TF/init_state{n_qubits}.npy")

In [1384]:
# Z observable and I/2
local_obs = np.array([[1., 0.], [0., -1.]])
identity_ent = np.eye(2)/2
identity_total = np.eye(2**n_qubits_total)

In [1385]:
def get_depol_ops(site):
    sx = np.kron(np.eye(2**site), np.kron(np.array([[0., 1.], [1., 0.]]), np.eye(2**(n_qubits_total - site - 1))))
    sy = np.kron(np.eye(2**site), np.kron(np.array([[0., -1j], [1j, 0.]]), np.eye(2**(n_qubits_total - site - 1))))
    sz = np.kron(np.eye(2**site), np.kron(np.array([[1., 0.], [0., -1.]]), np.eye(2**(n_qubits_total - site - 1))))
    return sx, sy, sz

In [1386]:
def ptrace(mtx, hold):
    """Performs partial trace of state with only one site #hold remains"""
    indexes = list(range(2*n_qubits_total))
    axis_order = [indexes.pop(hold), indexes.pop(hold + n_qubits_total - 1)]
    axis_order = indexes + axis_order
    mtx = np.transpose(np.reshape(mtx, [2]*2*n_qubits_total), axis_order)
    return np.trace(np.reshape(mtx, (2**(n_qubits_total-1), 2**(n_qubits_total-1), 2, 2)))

def evolution_step(state):
    """Simple application of evolution step propagator"""
    state = propagator @ state @ propagator_dag
    # assert np.abs(np.trace(state) - 1.) < tp_tol, f"Trace is not preserved.{np.trace(state)}"
    return state / np.trace(state)

def evolution_step_obs(obs):
    """Simple application of evolution step propagator"""
    obs = propagator_dag @ obs @ propagator
    return obs

In [1387]:
depol_ops = [get_depol_ops(site) for site in range(n_qubits_total)]

In [1388]:
# FOR TESTING NONSELECTIVE MEASUREMENTS DONT FORGET TO CHANGE PARAMETER rand !!!!!


def entangled_channel_step(state, p, rand = True):
    """Application of entanglement channel with and depolarization channel with probability of depolarization 'p'"""
    if not rand:
        state = ent_transform @ state @ ent_transform_dag
    else:
        state = PB00 @ state @ PB00_dag + PB11 @ state @ PB11_dag + PB10 @ state @ PB10_dag + PB01 @ state @ PB01_dag
        
    if p != 0:
        for site in [0, 1, 4, 5]:
            state = (1 - p) * state + p/3 * sum([depol_ops[site][i] @ state @ depol_ops[site][i] for i in range(3)])
    return state / np.trace(state)

def entangled_channel_step_obs(state, p, rand = True):
    """Application of entanglement channel with and depolarization channel with probability of depolarization 'p'"""
    if not rand:
        state = ent_transform_dag @ state @ ent_transform
    else:
        state = PB00_dag @ state @ PB00 + PB11_dag @ state @ PB11 + PB10_dag @ state @ PB10 + PB01_dag @ state @ PB01
    if p != 0:
        for site in [0, 1, 4, 5]:
            state = (1 - p) * state + p/3 * sum([depol_ops[site][i] @ state @ depol_ops[site][i] for i in range(3)])
    return state

In [1389]:
tracing_krauss0 = np.concatenate([np.eye(2**(n_qubits_total - 1)), np.zeros((2**(n_qubits_total - 1), 2**(n_qubits_total - 1)))], axis=1)
tracing_krauss1 = np.concatenate([np.zeros((2**(n_qubits_total - 1), 2**(n_qubits_total - 1))), np.eye(2**(n_qubits_total - 1))], axis=1)

In [1390]:
def get_encoding_krauss(inp):
    tensor_product_krauss = np.eye(2**(n_qubits_total - 1)).ravel()
    tensor_product_krauss = np.stack([tensor_product_krauss * np.sqrt(inp), tensor_product_krauss * np.sqrt(1. - inp)]).reshape(2**(n_qubits_total), 2**(n_qubits_total-1))
    return tensor_product_krauss

In [1391]:
def encoding_step(state, inp):
    """Encoding of inp to reservoir site #0"""
    # Partial trace. #0 site is traced out
#     axis_order = [0, n_qubits_total] + list(range(1, n_qubits_total)) + list(
#         range(n_qubits_total + 1, 2*n_qubits_total))
#     traced_dm = np.trace(np.reshape(
#         np.transpose(np.reshape(state, [2]*2*n_qubits_total), axis_order),
#         (2, 2, 2**(n_qubits_total-1), 2**(n_qubits_total-1))
#     ))
#     # Encoding
#     encoding_node_state = np.array([[np.sqrt(inp), np.sqrt(1. - inp)]])
#     return np.kron(encoding_node_state.T @ encoding_node_state, traced_dm)
    enc_krauss = get_encoding_krauss(inp)
    state = tracing_krauss0 @ state @ tracing_krauss0.T + tracing_krauss1 @ state @ tracing_krauss1.T
    state = enc_krauss @ state @ enc_krauss.T
    return state

In [1392]:
def separated_measurement_step(state, site):
    """Nakajima pipeline"""
    # Qubit #site dm and Z observable application
    one_qubit_dm = ptrace(state, site)
    outcome = np.real(np.trace(local_obs @ one_qubit_dm))
    return state, outcome

In [1393]:
def get_mutual_information(state):
    axis_order = list(range(n_qubitsA, n_qubits_total)) + list(range(n_qubitsA + n_qubits_total, 2*n_qubits_total)) + list(
        range(0, n_qubitsA)) + list(range(n_qubits_total, n_qubitsA + n_qubits_total))
    dmA = np.trace(np.reshape(
        np.transpose(np.reshape(state, [2]*2*n_qubits_total), axis_order),
        (2**n_qubitsB, 2**n_qubitsB, 2**n_qubitsA, 2**n_qubitsA)
    ))
    axis_order = list(range(0, n_qubitsA)) + list(range(n_qubits_total, n_qubitsA + n_qubits_total)) + list(
        range(n_qubitsA, n_qubits_total)) + list(range(n_qubitsA + n_qubits_total, 2*n_qubits_total))
    dmB = np.trace(np.reshape(
        np.transpose(np.reshape(state, [2]*2*n_qubits_total), axis_order),
        (2**n_qubitsA, 2**n_qubitsA, 2**n_qubitsB, 2**n_qubitsB)
    ))
    eigsS, _ = np.linalg.eigh(state)
    eigsA, _ = np.linalg.eigh(dmA)
    eigsB, _ = np.linalg.eigh(dmB)
    return entropy(eigsS[eigsS > tp_tol]) - entropy(eigsA[eigsA > tp_tol]) - entropy(eigsB[eigsB > tp_tol])

**KRAUSS OPS TESTED**

###################################################################

In [1394]:
state = evolution_step_obs(init_state)
indexes = list(range(2*n_qubits_total))
axis_order = [indexes.pop(0), indexes.pop(n_qubits_total - 1)]
axis_order = axis_order + indexes
state1 = np.transpose(np.reshape(state, [2]*2*n_qubits_total), axis_order)
state1 = np.trace(np.reshape(state1, (2, 2, 2**(n_qubits_total-1), 2**(n_qubits_total-1))))

state2 = tracing_krauss0 @ state @ tracing_krauss0.T + tracing_krauss1 @ state @ tracing_krauss1.T

assert np.linalg.norm(state2 - state1) < 1e-12, "Tracing ops failure in value"
assert state2.shape == (2**(n_qubits_total-1), 2**(n_qubits_total-1)), "Tracing ops failure in shape"

In [1395]:
inp = 0.5
tensor_product_krauss = get_encoding_krauss(inp)
state = evolution_step(init_state)
state1 = encoding_step(state, inp)
state2 = tensor_product_krauss @ (tracing_krauss0 @ state @ tracing_krauss0.T + tracing_krauss1 @ state @ tracing_krauss1.T) @ tensor_product_krauss.T
assert np.linalg.norm(state2 - state1) < 1e-12, "Tensor product ops failure in value"
assert state2.shape == (2**n_qubits_total, 2**n_qubits_total), "Tensor product ops failure in shape"

###################################################################

In [1396]:
# signal = np.load('/content/gdrive/My Drive/QRC/data/complex_signal_9_9.npy')
#signal = np.load(path_main + '/data/complex_signal_9_9.npy')
signal = [np.random.choice(2) for _ in range(2200)]
inp = signal[2000:2200]
multiplexing = 10

In [1397]:
init_obs = [depol_ops[site][2] for site in range(1, n_qubits_total)]

In [1398]:
def get_obs_at_time(obs, timestep, dp):
    for n in range(timestep):
        op = get_encoding_krauss(inp[timestep-n])
        obs = op.T @ obs @ op
        obs = tracing_krauss0.T @ obs @ tracing_krauss0 + tracing_krauss1.T @ obs @ tracing_krauss1
        for m in range(multiplexing):
            obs = entangled_channel_step_obs(obs, dp)
            obs = evolution_step_obs(obs)
    return obs

In [1399]:
def evaluate(depolarization_prob):
    # Initialization
    otoc = np.zeros((len(inp), n_qubits_total - 1))
    mutual_information = np.zeros(len(inp))
    result = np.zeros((len(inp), multiplexing, n_qubitsB))
    purity = np.zeros(len(inp))
    state = init_state
    
    for i in range(2000):
        state = encoding_step(state, signal[i])
        state = evolution_step(state)
        # CHANGE rand TO TRUE FOR NONSELECTIVE MEASURMENTS SETUP !!!
        state = entangled_channel_step(state, depolarization_prob)
    for n in tqdm(range(len(inp))):
        for idx, obs in enumerate(init_obs):
            obs_at_time = get_obs_at_time(obs, n, depolarization_prob)
            otoc[n, idx] = np.real(np.trace(state @ obs @ obs_at_time @ obs @ obs_at_time))
    for n in tqdm(range(len(inp))):
        purity[n] = np.real(np.trace(state @ state))
        # Encoding
        state = encoding_step(state, inp[n])
        # Main part
        for m in range(multiplexing):
            # Evolution -> Entanglement -> Evolution
            state = evolution_step(state)
            # CHANGE rand TO TRUE FOR NONSELECTIVE MEASURMENTS SETUP !!!
            state = entangled_channel_step(state, depolarization_prob)

            for site in range(n_qubitsB):
                state, outcome = separated_measurement_step(state, site + n_qubitsA)
                result[n, m, site] = outcome
        
        mutual_information[n] = get_mutual_information(state)

    return result, mutual_information, otoc, purity, state

In [1400]:
# p = 0
# results, mutual_information, otoc, purity, state = evaluate(p)

p = [0, 0.001, 0.01, 0.1, 0.5]
for prob in p:
    results, mutual_information, otoc, purity, state = evaluate(prob)
    np.save(path_CHAOS_SIGNAL + 'OTOC_p' + str(prob), otoc)
    np.save(path_CHAOS_SIGNAL + 'results_p' + str(prob), results)
    np.save(path_CHAOS_SIGNAL + 'state_p' + str(prob), state)
    np.save(path_CHAOS_SIGNAL + 'purity_p' + str(prob), purity)
    np.save(path_CHAOS_SIGNAL + 'mutual_information_p' + str(prob), mutual_information)


In [ ]:
fig = plt.figure(figsize = (32, 16))
font = font_manager.FontProperties(family='Times New Roman',
                                   weight='bold',
                                   style='normal', size=25)
font1 = {'family': 'Times New Roman',
        'weight': 'normal',
        'size': 50,
        }
ax = plt.subplot(111)
ax.tick_params(axis='both', which='major', labelsize=45, width = 3, length = 10, direction = 'inout')

#for i in range(5):
#    plt.plot(otoc[:20, i], label = 'Z' + str(i))
    
plt.plot(otoc[:20,0], label = 'A:Z' + str(2), linewidth = 4)
plt.plot(otoc[:20,1], label = 'Bridge A:Z' + str(3), linewidth = 4)
plt.plot(otoc[:20,2], label = 'Bridge B:Z' + str(4),  linewidth = 4)

plt.plot(otoc[:20,3], label = 'B:Z' + str(5), linewidth = 4)
plt.plot(otoc[:20,4], label = 'B:Z' + str(6),  linewidth = 4)

plt.plot(np.mean(otoc[:20], axis=-1), label = 'Mean',  linewidth = 4)
    
plt.legend(frameon=False,loc='upper right', prop = font1)

fig.savefig(path_CHAOS_SIGNAL + 'OTOC' + '.svg', format='svg', dpi=3000)

In [ ]:
def learning(results, offset):
    y_true = np.array(signal[2000-offset:2200-offset]) * 2 - 1
    X = results.reshape(-1, (n_qubitsB) * multiplexing)
    X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
    W = np.linalg.inv(X.T @ X + 1e-7*np.eye(X.shape[1])) @ X.T @ y_true
    return X @ W, y_true

def learning_gen(results, offset):
    y_true = np.array(signal[2000-offset:2100-offset]) * 2 - 1
    y_test = np.array(signal[2100-offset:2200-offset]) * 2 - 1
    X = results.reshape(-1, (n_qubitsB) * multiplexing)
    X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
    X_train = X[:100]
    X_test = X[100:]
    W = np.linalg.inv(X_train.T @ X_train + 1e-18 *np.eye(X_train.shape[1])) @ X_train.T @ y_true
    return X_test @ W, y_test

In [ ]:
# LEARNING EVALUATION

res, y_true = learning(results, 0)
fig = plt.figure(figsize = (32, 16))
font = font_manager.FontProperties(family='Times New Roman',
                                   weight='bold',
                                   style='normal', size=10)
font1 = {'family': 'Times New Roman',
        'weight': 'normal',
        'size': 50,
        }
ax = plt.subplot(111)
ax.tick_params(axis='both', which='major', labelsize=40, width = 3, length = 10, direction = 'inout')
ax.set_ylim(-1.5, 1.7)
plt.plot(y_true, label = 'Target', linewidth=5, linestyle = '--', color = '#005394')
plt.plot(res, label = 'Learning',linewidth=4, color = '#C54C36')
leg = plt.legend(frameon=False, loc=(0.8,0.83), prop = font1)
leg_lines = leg.get_lines()
leg_texts = leg.get_texts()
plt.setp(leg_lines, linewidth=5)

fig.savefig(path_CHAOS_SIGNAL + 'LEARNING' + '.svg', format='svg', dpi=3000)

In [ ]:
# GENERALIZATION

offset = 3
res_gen, y_true_gen = learning_gen(results, offset)

fig = plt.figure(figsize = (32, 16))
font = font_manager.FontProperties(family='Times New Roman',
                                   weight='bold',
                                   style='normal', size=10)
font1 = {'family': 'Times New Roman',
        'weight': 'normal',
        'size': 50,
        }
ax = plt.subplot(111)
ax.tick_params(axis='both', which='major', labelsize=40, width = 3, length = 10, direction = 'inout')
ax.set_ylim(-1.5, 1.8)

plt.plot(y_true_gen, label = 'Target', linewidth=5, linestyle = '--', color = '#005394')
plt.plot(res_gen, label = 'Prediction',linewidth=5, color = '#C54C36')
leg = plt.legend(frameon=False, loc=(0.79,0.83), prop = font1)
leg_lines = leg.get_lines()
leg_texts = leg.get_texts()
plt.setp(leg_lines, linewidth=5)


fig.savefig(path_CHAOS_SIGNAL + 'PREDICTION_offset_'+ str(offset) + '.svg', format='svg', dpi=3000)




In [ ]:
result_list = [np.load(path_CHAOS_SIGNAL + 'results_p' + str(prob)) + '.npy' for prob in p ]

In [ ]:

def get_memory_acc(result):
    acc = []
    for i in range(50):
        res, y_true = learning(result, i)
        acc.append((np.mean(res * y_true) - np.mean(res) * np.mean(y_true))**2/np.var(res)/np.var(y_true))
    return acc


def learn_acc(d_max, result):
    acc = []
    for d in range(d_max):
        res, y_true = learning_gen(result, d)
        #acc.append(np.linalg.norm(res - y_true,ord = 'fro'))
        acc.append(np.sum(res - y_true)**2)
    return np.array(acc)

mem_acc_list = []
mem_cap_list = []
learning_acc_list = []

for results in result_list:
    memory_acc = get_memory_acc(results)
    memory_cap = np.array([np.mean(memory_acc[0: d]) for d in range(1, len(memory_acc))])
    learning_acc = learn_acc(20,results)
    
    mem_acc_list.append(memory_acc)
    mem_cap_list.append(memory_cap)
    learning_acc_list.append(learning_acc)




In [ ]:


fig, ax = plt.subplots(3, figsize = (16,32))
#fig = plt.figure(figsize = (32, 16))
font = font_manager.FontProperties(family='Times New Roman',
                                   weight='bold',
                                   style='normal', size=25)
font1 = {'family': 'Times New Roman',
        'weight': 'normal',
        'size': 50,
        }

ax[0].tick_params(axis='both', which='major', labelsize=30, width = 3, length = 10, direction = 'inout')
ax[1].tick_params(axis='both', which='major', labelsize=30, width = 3, length = 10, direction = 'inout')
ax[2].tick_params(axis='both', which='major', labelsize=30, width = 3, length = 10, direction = 'inout')


ax[0].plot(memory_acc, label = 'Memory Acc: p = 0', linewidth = 5, color = '#005394')
ax[0].plot(memory_acc, label = 'Memory Acc: p = 0.01', linewidth = 5, color = '#C54C36')
ax[0].plot(memory_acc, label = 'Memory Acc: p = 0.1', linewidth = 5, color = '#74C265')
ax[0].legend(loc = 'upper right', fontsize=30, prop = font1)

ax[1].plot(memory_cap, label = 'Memory Cap.p = 0', linewidth = 5, color = '#005394')
ax[1].plot(memory_cap, label = 'Memory Cap.p = 0.01', linewidth = 5, color = '#C54C36')
ax[1].plot(memory_cap, label = 'Memory Cap.p = 0.1', linewidth = 5, color = '#74C265')
ax[1].legend(loc = 'upper right', fontsize=30,  prop = font1)



ax[2].plot(learn_acc(20), label = 'Gen. Acc: p = 0', linewidth = 3, linestyle = '--',  marker = 'o',markersize = 20, color = '#005394')
ax[2].plot(learn_acc(20), label = 'Gen. Acc: p =0.01', linewidth = 3, linestyle = '--',  marker = 's',markersize = 20, color = '#C54C36')
ax[2].plot(learn_acc(20), label = 'Gen. Acc: p =0.1', linewidth = 3, linestyle = '--',  marker = '^',markersize = 20, color = '#74C265')
ax[2].legend(loc = 'upper left', fontsize=30,  prop = font1)

fig.savefig(path_CHAOS_SIGNAL + 'TASKS_BENCHMARKS' + '.svg', format='svg', dpi=3000)